### This Notebook show the flow of calls to use the SQL Agent + routing
* Text2SQL (this NB is using the ai_sql_agent)
* Routing

In [1]:
import os
import sys

# Set the PYTHONPATH environment variable
os.environ['PYTHONPATH'] = '..'

# Add it to sys.path so that it's included in the Python import path
sys.path.append(os.environ['PYTHONPATH'])

In [2]:
from oci_cohere_embeddings_utils import OCIGenAIEmbeddingsWithBatch

# only for showing the table of rows in this NB
import pandas as pd
from decimal import Decimal

from langchain_core.prompts import PromptTemplate
from database_manager import DatabaseManager
from llm_manager import LLMManager
from schema_manager_23ai import SchemaManager23AI
from ai_sql_agent import AISQLAgent
from router import Router
from core_functions import generate_sql_with_models
from utils import get_console_logger
from prompt_template import PROMPT_TEMPLATE
from prompt_routing import PROMPT_CHAT_ON_DATA

from config import (
    AUTH_TYPE,
    CONNECT_ARGS,
    MODEL_LIST,
    MODEL_ENDPOINTS,
    TEMPERATURE,
    DEBUG,
    EMBED_MODEL_NAME,
    EMBED_ENDPOINT,
    INDEX_MODEL_FOR_EXPLANATION
)
from config_private import COMPARTMENT_OCID

/Users/lsaetta/miniforge3/envs/text2sql/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Setup 

In [3]:
logger = get_console_logger()

db_manager = DatabaseManager(CONNECT_ARGS, logger)

ai_sql_agent = AISQLAgent(
        CONNECT_ARGS,
        MODEL_LIST,
        MODEL_ENDPOINTS,
        COMPARTMENT_OCID,
        EMBED_MODEL_NAME,
        EMBED_ENDPOINT,
        TEMPERATURE,
        PROMPT_TEMPLATE,
    )

# for the router
llm_manager = LLMManager(
    MODEL_LIST, MODEL_ENDPOINTS, COMPARTMENT_OCID, TEMPERATURE, logger
)

router = Router(llm_manager)

2024-10-02 14:05:43,104 - Connecting to the Database...
2024-10-02 14:05:43,136 - DB engine created...
2024-10-02 14:05:43,136 - Connecting to the Database...
2024-10-02 14:05:43,137 - DB engine created...
2024-10-02 14:05:43,137 - LLMManager: Initialising the list of models...
2024-10-02 14:05:43,138 - Model: meta.llama-3.1-70b-instruct
2024-10-02 14:05:43,273 - Model: cohere.command-r-plus
2024-10-02 14:05:43,340 - Model: meta.llama-3.1-405b-instruct
2024-10-02 14:05:43,470 - Loading Schema Manager...
2024-10-02 14:05:43,471 - AI SQL Agent initialized successfully.
2024-10-02 14:05:43,471 - LLMManager: Initialising the list of models...
2024-10-02 14:05:43,471 - Model: meta.llama-3.1-70b-instruct
2024-10-02 14:05:43,537 - Model: cohere.command-r-plus
2024-10-02 14:05:43,602 - Model: meta.llama-3.1-405b-instruct


#### The user request

In [9]:
user_request = "List the top 20 sales by total amount, with product name, customer name, country name for sales in Europe"

#### Generate the SQL

In [10]:
%%time
sql_query = ai_sql_agent.generate_sql_query(user_request, user_group_id=None)

2024-10-02 14:06:15,956 - Generating restricted schema for user request...
2024-10-02 14:06:17,427 - Identifying relevant tables for query...
2024-10-02 14:06:17,429 - - COUNTRIES
2024-10-02 14:06:17,430 - - SALES
2024-10-02 14:06:17,431 - - PRODUCTS
2024-10-02 14:06:17,434 - - CHANNELS
2024-10-02 14:06:17,437 - - CUSTOMERS
2024-10-02 14:06:17,438 - - HR_COUNTRIES
2024-10-02 14:06:21,558 - Reranker result:
2024-10-02 14:06:21,559 - ['SALES', 'PRODUCTS', 'CUSTOMERS', 'COUNTRIES']
2024-10-02 14:06:21,561 - 
2024-10-02 14:06:21,601 - Restricted schema generated.
2024-10-02 14:06:21,602 - Generating SQL query...
2024-10-02 14:06:24,919 - SQL query generated.


CPU times: user 130 ms, sys: 20.2 ms, total: 151 ms
Wall time: 8.96 s


In [11]:
print(sql_query)

SELECT 
    p.prod_name, 
    c.cust_first_name || ' ' || c.cust_last_name AS customer_name, 
    co.country_name, 
    s.amount_sold
FROM 
    sales s
JOIN 
    products p ON s.prod_id = p.prod_id
JOIN 
    customers c ON s.cust_id = c.cust_id
JOIN 
    countries co ON c.country_id = co.country_id
WHERE 
    co.country_region = 'Europe'
ORDER BY 
    s.amount_sold DESC
FETCH FIRST 20 ROWS ONLY


#### Test the SQL query

In [12]:
rows = db_manager.execute_sql(sql_query)

2024-10-02 14:06:36,022 - Found 20 rows..


In [13]:
# use Pandas Dataframe for quick display
df = pd.DataFrame(rows)

df

,amount_sold,country_name,customer_name,prod_name
0,1782.72,United Kingdom,Zoe Ballanger,Envoy Ambassador
1,1782.72,Spain,Trevor Manson,Envoy Ambassador
2,1782.72,Denmark,Chadwick Klemm,Envoy Ambassador
3,1782.72,United Kingdom,Zillah Driscoll,Envoy Ambassador
4,1782.72,United Kingdom,Isabel Baley,Envoy Ambassador
5,1782.72,Spain,Ines Lovett,Envoy Ambassador
6,1782.72,Germany,Brett Kugel,Envoy Ambassador
7,1782.72,United Kingdom,Antony Chinn,Envoy Ambassador
8,1782.72,Germany,Harrie Rider,Envoy Ambassador
9,1782.72,Germany,Buddy Chen,Envoy Ambassador


#### Test **Routing** and **Chat on Data**

In [14]:
# setup model for data analysis
# this is is only used for data analysis
analyze_prompt = PromptTemplate.from_template(PROMPT_CHAT_ON_DATA)

# get llm to be used
llm_c = llm_manager.llm_models[
            INDEX_MODEL_FOR_EXPLANATION]

analyze_chain = analyze_prompt | llm_c

In [19]:
# requests

first_request = "List the top 20 sales by total amount, with product name, customer name, country name for sales in Europe"
second_request = "Analyze the data retrieved. Create a report."
third_request = "Generate another report on the data. I need a breakdown for country."

# this is the list of request that will be processed
requests_list = [first_request, second_request, third_request]

In [20]:
# dispatching logic
def generate_sql_handler(request):
    sql_query = ai_sql_agent.generate_sql_query(request, user_group_id=None)
    data = db_manager.execute_sql(sql_query)

    # display data
    df = pd.DataFrame(data)
    print(df)
    print("")
    return data  # Returning data to use in further actions if needed


def analyze_data_handler(request, data):
    print("Analyzing data...")

    result = analyze_chain.invoke({"data": data, "question": request})

    print("")
    print("Analysis of data:")
    print(result.content)
    print("")


def not_defined_handler(request):
    print("Please add more details on what you want me to do..")


# Create a dispatch table (dictionary) for classification types
dispatch_table = {
    "generate_sql": generate_sql_handler,
    "analyze_data": analyze_data_handler,
    "not_defined": not_defined_handler,
}

In [22]:
for request in requests_list:
    print("")
    print("request is: ", request)

    # classify the request using the router
    classification = router.classify(request)
    print("request classified as: ", classification)

    # do based on classification type
    if classification in dispatch_table:
        # 'generate_sql_handler' returns data, and we pass it to 'analyze_data_handler' if needed
        if classification == "generate_sql":
            data = dispatch_table[classification](request)  # Get data to reuse
        elif classification == "analyze_data":
            dispatch_table[classification](request, data)
        else:
            dispatch_table[classification](request)
    else:
        print("Invalid classification provided.")


request is:  List the top 20 sales by total amount, with product name, customer name, country name for sales in Europe


2024-10-02 14:18:13,047 - Generating restricted schema for user request...


request classified as:  generate_sql


2024-10-02 14:19:20,116 - Identifying relevant tables for query...
2024-10-02 14:19:20,118 - - COUNTRIES
2024-10-02 14:19:20,118 - - SALES
2024-10-02 14:19:20,122 - - PRODUCTS
2024-10-02 14:19:20,124 - - CHANNELS
2024-10-02 14:19:20,124 - - CUSTOMERS
2024-10-02 14:19:20,125 - - HR_COUNTRIES
2024-10-02 14:20:32,302 - Reranker result:
2024-10-02 14:20:32,304 - ['SALES', 'PRODUCTS', 'CUSTOMERS', 'COUNTRIES']
2024-10-02 14:20:32,305 - 
2024-10-02 14:20:32,341 - Restricted schema generated.
2024-10-02 14:20:32,344 - Generating SQL query...
2024-10-02 14:21:23,541 - SQL query generated.
2024-10-02 14:21:34,067 - Found 20 rows..


   amount_sold    country_name     customer_name         prod_name
0      1782.72  United Kingdom     Zoe Ballanger  Envoy Ambassador
1      1782.72           Spain     Trevor Manson  Envoy Ambassador
2      1782.72         Denmark    Chadwick Klemm  Envoy Ambassador
3      1782.72  United Kingdom   Zillah Driscoll  Envoy Ambassador
4      1782.72  United Kingdom      Isabel Baley  Envoy Ambassador
5      1782.72           Spain       Ines Lovett  Envoy Ambassador
6      1782.72         Germany       Brett Kugel  Envoy Ambassador
7      1782.72  United Kingdom      Antony Chinn  Envoy Ambassador
8      1782.72         Germany      Harrie Rider  Envoy Ambassador
9      1782.72         Germany        Buddy Chen  Envoy Ambassador
10     1782.72         Germany    Hannah Feldman  Envoy Ambassador
11     1782.72  United Kingdom    Goddard Remler  Envoy Ambassador
12     1782.72           Italy     Desma Janssen  Envoy Ambassador
13     1782.72         Germany  Hannah Gutierrez  Envoy Ambass